-----------
# Forescast - Séries Temporais com Chronos da Amazon

--------

<div align="center">
<img src="https://raw.githubusercontent.com/amazon-science/chronos-forecasting/main/figures/chronos-logo.png" width="30%">
</div>

<div align="center">

</div>

## Introdução

Chronos é uma família de **modelos pré-treinados de previsão de séries temporais** baseados em arquiteturas de modelos de linguagem.  
Uma série temporal é transformada em uma sequência de tokens por meio de escalonamento e quantização. O modelo de linguagem é treinado nesses tokens usando perda de entropia cruzada.  

Após o treinamento, previsões probabilísticas são obtidas amostrando múltiplas trajetórias futuras dadas as observações históricas.  
Os modelos Chronos foram treinados em um grande corpus de séries temporais públicas, além de dados sintéticos gerados via processos Gaussianos.

Leia o artigo para mais detalhes: [Chronos: Learning the Language of Time Series](https://arxiv.org/abs/2403.07815).

<p align="center">
  <img src="https://raw.githubusercontent.com/amazon-science/chronos-forecasting/main/figures/main-figure.png" width="50%">
  <br />
  <span>
    Fig. 1: Visão geral do Chronos. (<b>Esquerda</b>) A série temporal é escalada e quantizada em tokens.  
    (<b>Centro</b>) Os tokens são processados por um modelo de linguagem (encoder-decoder ou apenas decoder), treinado com perda de entropia cruzada.  
    (<b>Direita</b>) Na inferência, amostramos tokens de forma autoregressiva e mapeamos de volta para valores numéricos.  
    Várias trajetórias são geradas para formar uma distribuição preditiva.
  </span>
</p>




### Histórico de lançamentos dos modelos
- **14 Fev 2025**: Chronos-Bolt agora está disponível no Amazon SageMaker JumpStart! Confira o [notebook tutorial](notebooks/deploy-chronos-bolt-to-amazon-sagemaker.ipynb) para aprender a implantar endpoints Chronos em produção em apenas 3 linhas de código.
- **12 Dez 2024**: Lançamos [`fev`](https://github.com/autogluon/fev), um pacote leve para benchmark de modelos de previsão de séries temporais baseado na biblioteca [Hugging Face `datasets`](https://huggingface.co/docs/datasets/en/index).
- **26 Nov 2024**: Modelos Chronos-Bolt lançados no [HuggingFace](https://huggingface.co/collections/amazon/chronos-models-65f1791d630a8d57cb718444). Eles são **5% mais precisos**, até **250x mais rápidos** e **20x mais eficientes em memória** do que os modelos originais de mesmo porte!
- **27 Jun 2024**: [Lançamos os datasets](https://huggingface.co/datasets/autogluon/chronos_datasets) usados no artigo e um [script de avaliação](./scripts/README.md#evaluating-chronos-models) para calcular as métricas WQL e MASE.
- **17 Mai 2024**:  Corrigimos um erro de "off-by-one" nos índices de bin no `output_transform`. Essa correção simples melhorou significativamente o desempenho geral do Chronos.
- **10 Mai 2024**:  Adicionamos o código para pré-treinamento e fine-tuning dos modelos Chronos. Você pode encontrá-lo [nesta pasta](./scripts/training). Também adicionamos um [script](./scripts/kernel-synth.py) para gerar séries temporais sintéticas a partir de processos Gaussianos (KernelSynth; veja a Seção 4.2 do artigo).
- **19 Abr 2024**:  Chronos agora é suportado no [AutoGluon-TimeSeries](https://auto.gluon.ai/stable/tutorials/timeseries/index.html), um pacote AutoML poderoso para previsão de séries temporais com suporte a ensembles, deploy em nuvem e muito mais. Comece com o [tutorial](https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html).
- **08 Abr 2024**:  Suporte experimental a [inferência MLX](https://github.com/amazon-science/chronos-forecasting/tree/mlx) adicionado para Macs com Apple Silicon. Isso permite previsões bem mais rápidas em comparação ao CPU.
- **25 Mar 2024**:  [Versão 1.1.0 lançada](https://github.com/amazon-science/chronos-forecasting/releases/tag/v1.1.0) com otimizações de inferência e `pipeline.embed` para extrair embeddings do encoder.
- **13 Mar 2024**:  Artigo [Chronos](https://arxiv.org/abs/2403.07815) e código de inferência lançados.


### Arquitetura

Os modelos deste repositório são baseados na arquitetura [T5](https://arxiv.org/abs/1910.10683).  
A diferença é o tamanho do vocabulário: os modelos Chronos-T5 usam **4096 tokens** (contra **32128** do T5 original), o que reduz o número de parâmetros.

<div align="center">

| Modelo                                                                 | Parâmetros | Baseado em                                                             |
| ---------------------------------------------------------------------- | ---------- | ---------------------------------------------------------------------- |
| [**chronos-t5-tiny**](https://huggingface.co/amazon/chronos-t5-tiny)   | 8M         | [t5-efficient-tiny](https://huggingface.co/google/t5-efficient-tiny)   |
| [**chronos-t5-mini**](https://huggingface.co/amazon/chronos-t5-mini)   | 20M        | [t5-efficient-mini](https://huggingface.co/google/t5-efficient-mini)   |
| [**chronos-t5-small**](https://huggingface.co/amazon/chronos-t5-small) | 46M        | [t5-efficient-small](https://huggingface.co/google/t5-efficient-small) |
| [**chronos-t5-base**](https://huggingface.co/amazon/chronos-t5-base)   | 200M       | [t5-efficient-base](https://huggingface.co/google/t5-efficient-base)   |
| [**chronos-t5-large**](https://huggingface.co/amazon/chronos-t5-large) | 710M       | [t5-efficient-large](https://huggingface.co/google/t5-efficient-large) |
| [**chronos-bolt-tiny**](https://huggingface.co/amazon/chronos-bolt-tiny)   | 9M         | [t5-efficient-tiny](https://huggingface.co/google/t5-efficient-tiny)   |
| [**chronos-bolt-mini**](https://huggingface.co/amazon/chronos-bolt-mini)   | 21M        | [t5-efficient-mini](https://huggingface.co/google/t5-efficient-mini)   |
| [**chronos-bolt-small**](https://huggingface.co/amazon/chronos-bolt-small) | 48M        | [t5-efficient-small](https://huggingface.co/google/t5-efficient-small) |
| [**chronos-bolt-base**](https://huggingface.co/amazon/chronos-bolt-base)   | 205M       | [t5-efficient-base](https://huggingface.co/google/t5-efficient-base)   |

</div>

-------------
**Veja como utilizar o Chronos** 

https://github.com/amazon-science/chronos-forecasting

https://auto.gluon.ai/stable/install.html

-------------
## **Chronos (T5/Bolt) — visão intuitiva**

#### Ideia central
- É um “autocomplete de números”: lê o histórico da série e completa o futuro passo a passo.
- Pré-treinado em milhões de séries → já “sabe” padrões gerais (tendências, sazonalidades, picos/vales).

#### Como o modelo enxerga a série
- Converte valores para uma representação estável (normalização interna).
- Trata a sequência como texto: usa uma janela recente (contexto) e aprende a continuar a série.

#### Como prevê (quantis)
- Produz **distribuições** futuras, das quais extraímos **quantis**:
  - **P50**: mediana (melhor chute).
  - **P10 / P90**: limites inferior/superior → **intervalo de projeção** (incerteza).
- Para horizontes longos, prevê em **blocos**, encadeando os resultados.

#### T5 vs. Bolt (em termos práticos)
- **T5**: mais pesado/tradicional na família Transformer.
- **Bolt**: **muito mais rápido** (funciona bem em CPU) com qualidade similar.
- Na prática, **Bolt** costuma ser o ponto de partida para velocidade + qualidade.

#### Pontos fortes
- Não exige estacionar série (dispensa diferenciação manual).
- Lida com faltas razoavelmente bem.
- Entrega **incerteza** via intervalos (P10–P90 etc.).
- Funciona em **zero-shot** (teste rápido com dados históricos).

#### Limites
- **Mudanças de regime** recentes e exógenas (sem “pistas” na própria série).
- **Histórico curto** → maior incerteza.
- **Muito longo prazo** → erro acumulado e intervalos mais largos.

-----------------


## Análise de dados de consumo de energia

**Por que estes dados são bons?**  
Trabalharemos com séries **mensais** de consumo de energia do Brasil, um conjunto **aberto, oficial e atual** (EPE). Essas séries exibem **sazonalidade bem marcada** (padrões recorrentes ao longo do ano) e tendência de longo prazo, o que é ideal para modelos que capturam padrões temporais e **quantificam incerteza**. Em resumo: dados **confiáveis**, **granulares** (mensais) e **relevantes para aplicações reais** de planejamento e análise de demanda.

**Cenário do projeto**  
Vamos usar o histórico para **prever os próximos 60 meses (5 anos)**, produzindo uma projeção central (P50) e intervalos (ex.: P10–P90) para apoiar decisões sob incerteza.

**Fonte dos dados**  
EPE – Empresa de Pesquisa Energética (dados abertos):  
https://www.epe.gov.br/pt/publicacoes-dados-abertos/publicacoes/consumo-de-energia-eletrica

<iframe title="Report Section" width="1200" height="600" src="https://app.powerbi.com/view?r=eyJrIjoiNzRkOWFhMmYtN2QwYy00ZmJkLTk0YzAtMzI5Nzk3NzYwY2JmIiwidCI6Ijk2YTIxZDAwLWQ4MTEtNGMwNS04YWM0LTJkNTcwODUyNmEwYSJ9" frameborder="0" allowfullscreen="true"></iframe>

### Links úteis
- **Energia Elétrica – Dados abertos (mensal)**:  
  https://www.epe.gov.br/pt/publicacoes-dados-abertos/dados-abertos/dados-do-consumo-mensal-de-energia-eletrica

**Observações**  
- Garantimos frequência **mensal (MS)** contínua e tratamento de eventuais lacunas.  
- As projeções serão avaliadas em período de teste com **métricas transparentes** (MAE, RMSE, MAPE, R²).  
- O foco é **aplicação prática**: insumos para planejamento de capacidade, estudos de mercado e análise de políticas públicas.

----------

https://arxiv.org/abs/2403.07815

#### Instalar e Importar bibliotecas

In [ ]:
#!pip install chronos-forecasting   # instala a biblioteca Chronos (modelos de séries temporais da Amazon/HF)

import numpy as np                  # operações numéricas (vetores/matrizes, estatísticas rápidas)
import pandas as pd                 # DataFrames: leitura, limpeza, merge, reamostragem temporal
import torch                        # backend tensorial (CPU/GPU) utilizado pelo Chronos (PyTorch)

from chronos import BaseChronosPipeline  # pipeline do Chronos para carregar modelos e prever quantis (p10/p50/p90)

from sklearn.preprocessing import MinMaxScaler  # escala/normaliza dados (ex.: para [0,1]) e melhora estabilidade numérica

# (Estas três métricas também são importadas novamente abaixo; aqui é redundante)
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, median_absolute_error, r2_score
# MAE: erro absoluto médio |y-ŷ|; MSE: erro quadrático médio (costuma-se usar sqrt(MSE)=RMSE);
# MAPE: erro percentual médio (cuidado quando y=0).

import seaborn as sns              # visualização estatística/EDA com base no matplotlib (estilo e gráficos facilitados)
from datetime import datetime      # manipulação de timestamps (parse, formatação, diferenças)
import matplotlib.pyplot as plt    # visualização estática base (gráficos simples/rápidos)

import joblib                      # persistência de objetos (ex.: salvar scaler/modelos auxiliares em disco)
from pathlib import Path           # manipulação de caminhos de forma multiplataforma (mais seguro que strings)
import warnings                    # controle de avisos/alerts do Python e libs
warnings.filterwarnings('ignore')  # suprime warnings no output (limpa logs; use com parcimônia)

import random                      # gerador pseudoaleatório (seed para reprodutibilidade em partes estocásticas)
import os                          # utilidades do sistema (variáveis de ambiente, paths, etc.)
import unidecode                   # normaliza strings removendo acentos (útil para nomes de colunas/arquivos)

import plotly.graph_objects as go  # gráficos interativos (linhas/áreas); ótimo para visualizar previsão + intervalos
from plotly.subplots import make_subplots  # cria figuras com múltiplos subplots (quando precisar painéis lado a lado)

# --------------------------- SEED / REPRODUTIBILIDADE ---------------------------
SEED = 2024                         # valor da semente para resultados reprodutíveis
np.random.seed(SEED)                # fixa aleatoriedade do NumPy
random.seed(SEED)                   # fixa aleatoriedade do módulo random
os.environ['TF_DETERMINISTIC_OPS'] = '1'  # flag do TensorFlow (aqui é irrelevante; não usamos TF, pode remover)

# --------------------------- CAMINHOS / MODELOS OFFLINE -------------------------
# Usando raw string no Windows para evitar problemas com barras invertidas
Path_models = Path(r"D:/OneDrive/Documentos/GitHub/ciencia-dados-portifolio/Time_Series_Forecast_Chronos_Amazon")
# Caminho base para armazenar/carregar modelos localmente (modo OFFLINE).
# Para modo ONLINE (Hugging Face), use nomes com "/", ex.: "amazon/chronos-bolt-small", sem precisar deste path.
# salvar requirements.txt
!pip freeze > requirements.txt

### Importar dados de consumo de energia

In [24]:
# Importar dados direto do site
#df_bruto = pd.read_excel('https://www.epe.gov.br/_layouts/download.aspx?sourceURL=%2Fsites-pt%2Fpublicacoes-dados-abertos%2Fdados-abertos%2FDocuments%2FDados_abertos_Consumo_Mensal.xlsx')

# Importar dados local (mesmos do site)
df_bruto = pd.read_excel("dados/Dados_abertos_Consumo_Mensal.xlsx")

# Imformações
print(df_bruto.info())

# visualizar
df_bruto

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17688 entries, 0 to 17687
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Data            17688 non-null  int64         
 1   DataExcel       17688 non-null  datetime64[ns]
 2   Regiao          17688 non-null  object        
 3   Sistema         17688 non-null  object        
 4   Classe          17688 non-null  object        
 5   TipoConsumidor  17688 non-null  object        
 6   Consumo         17688 non-null  float64       
 7   Consumidores    17688 non-null  int64         
 8   DataVersao      17688 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 1.2+ MB
None


,Data,DataExcel,Regiao,Sistema,Classe,TipoConsumidor,Consumo,Consumidores,DataVersao
0,20250701,2025-07-01,Centro-Oeste,SUDESTE / CENTRO - OESTE,Comercial,Cativo,368351.667,496641,2025-08-25
1,20250701,2025-07-01,Centro-Oeste,SUDESTE / CENTRO - OESTE,Comercial,Livre,249881.102,3551,2025-08-25
2,20250701,2025-07-01,Centro-Oeste,SUDESTE / CENTRO - OESTE,Industrial,Cativo,62797.105,29804,2025-08-25
3,20250701,2025-07-01,Centro-Oeste,SUDESTE / CENTRO - OESTE,Industrial,Livre,903993.509,1330,2025-08-25
4,20250701,2025-07-01,Centro-Oeste,SUDESTE / CENTRO - OESTE,Outros,Cativo,298573.905,64345,2025-08-25
...,...,...,...,...,...,...,...,...,...
17683,20040101,2004-01-01,Sul,SUL,Industrial,Cativo,1464696.468,159190,2025-08-25
17684,20040101,2004-01-01,Sul,SUL,Industrial,Livre,346283.691,42,2025-08-25
17685,20040101,2004-01-01,Sul,SUL,Outros,Cativo,373249.407,84291,2025-08-25
17686,20040101,2004-01-01,Sul,SUL,Residencial,Cativo,1132506.000,6762504,2025-08-25


-------------------
### Preparar dados

Vamos consolidar **Cativo + Livre** para todas as classes (Residencial, Comercial, Industrial, Rural, Outros) e organizar o dataset para modelagem:

1. **Padronização de dimensões**
   - Região → **siglas** (`N`, `NE`, `CO`, `SE`, `S`)
   - Classe → **sem acento e minúscula** (ex.: `residencial`, `comercial`)

2. **Agregação**
   - Somar **`Consumo`** e **`Consumidores`** por **Data × Região × Classe**.

3. **Pivot amplo**
   - Transformar para formato **wide** com colunas por **(Região, Classe)** e métricas **Consumo/Consumidores**.

4. **Flatten de colunas**
   - Renomear para padrão:
     - `ec_<classe>_<sigla>` = **energia consumida**
     - `nc_<classe>_<sigla>` = **número de consumidores**

5. **Totais**
   - Criar **totais por região** (`ec_total_<sigla>`, `nc_total_<sigla>`) e **totais Brasil** (`ec_total_BR`, `nc_total_BR`).

6. **Higiene e saída**
   - Converter `Data` para `datetime` (mensal), ordenar e **salvar** em `dados/consumo_energia_pronto.xlsx`.

> Resultado: um **DataFrame pronto para modelagem**, com granularidade mensal e colunas padronizadas para consumo e número de consumidores por classe/região.


In [25]:
#================================================================================
# map de região 
#================================================================================
map_regiao = {
    "Norte": "N",
    "Nordeste": "NE",
    "Centro-Oeste": "CO",
    "Sudeste": "SE",
    "Sul": "S",
}

#================================================================================
# normalizar nomes de classe (sem acento, minúsculo)
#================================================================================
def norm_classe(x: str) -> str:
    return unidecode.unidecode(str(x)).lower().strip()

#================================================================================
# Agregar Cativo + Livre por Data, Região e Classe
#================================================================================
df_sum = (
    df_bruto.groupby(["Data", "Regiao", "Classe"], as_index=False)
            .agg({"Consumo": "sum", "Consumidores": "sum"})
)

#================================================================================
# Criar coluna de sigla da região e classe normalizada
#================================================================================
df_sum["RegiaoSigla"] = df_sum["Regiao"].map(map_regiao).fillna(df_sum["Regiao"])
df_sum["ClasseNorm"]  = df_sum["Classe"].apply(norm_classe)

#================================================================================
# Pivotar com colunas multi-index (RegiaoSigla, ClasseNorm)
#================================================================================
df_pivot = df_sum.pivot_table(
    index=["Data"],                                 # cada linha = Data
    columns=["RegiaoSigla", "ClasseNorm"],          # colunas = (Região, Classe)
    values=["Consumo", "Consumidores"],             # duas métricas
    aggfunc="sum",
    fill_value=0
)

#================================================================================
# Flatten dos nomes de coluna:
#================================================================================
#    ("Consumo", "SE", "comercial") -> ec_comercial_SE
#    ("Consumidores", "N",  "industrial") -> nc_industrial_N
def build_colname(metric, reg_sigla, classe):
    prefix = "ec" if metric == "Consumo" else "nc"
    return f"{prefix}_{classe}_{reg_sigla}"

df_pivot.columns = [
    build_colname(metric, reg, classe)
    for (metric, reg, classe) in df_pivot.columns.to_list()
]

#================================================================================
# Se quiser Data como coluna normal
#================================================================================
df_pivot = df_pivot.reset_index()

#================================================================================
## Gerar totais por região
#================================================================================

# Regiãoregião Norte (N)
df_pivot["ec_total_N"] = (
    df_pivot[["ec_residencial_N", "ec_comercial_N", "ec_industrial_N", "ec_rural_N", "ec_outros_N"]].sum(axis=1)
)
df_pivot["nc_total_N"] = (
    df_pivot[["nc_residencial_N", "nc_comercial_N", "nc_industrial_N", "nc_rural_N", "nc_outros_N"]].sum(axis=1)
)

# Região Nordeste (NE)
df_pivot["ec_total_NE"] = (
    df_pivot[["ec_residencial_NE", "ec_comercial_NE", "ec_industrial_NE", "ec_rural_NE", "ec_outros_NE"]].sum(axis=1)
)
df_pivot["nc_total_NE"] = (
    df_pivot[["nc_residencial_NE", "nc_comercial_NE", "nc_industrial_NE", "nc_rural_NE", "nc_outros_NE"]].sum(axis=1)
)

# Região Centro-Oeste (CO)
df_pivot["ec_total_CO"] = (
    df_pivot[["ec_residencial_CO", "ec_comercial_CO", "ec_industrial_CO", "ec_rural_CO", "ec_outros_CO"]].sum(axis=1)
)
df_pivot["nc_total_CO"] = (
    df_pivot[["nc_residencial_CO", "nc_comercial_CO", "nc_industrial_CO", "nc_rural_CO", "nc_outros_CO"]].sum(axis=1)
)

# Região Sudeste (SE)
df_pivot["ec_total_SE"] = (
    df_pivot[["ec_residencial_SE", "ec_comercial_SE", "ec_industrial_SE", "ec_rural_SE", "ec_outros_SE"]].sum(axis=1)
)
df_pivot["nc_total_SE"] = (
    df_pivot[["nc_residencial_SE", "nc_comercial_SE", "nc_industrial_SE", "nc_rural_SE", "nc_outros_SE"]].sum(axis=1)
)

# Região Sul (S)
df_pivot["ec_total_S"] = (
    df_pivot[["ec_residencial_S", "ec_comercial_S", "ec_industrial_S", "ec_rural_S", "ec_outros_S"]].sum(axis=1)
)
df_pivot["nc_total_S"] = (
    df_pivot[["nc_residencial_S", "nc_comercial_S", "nc_industrial_S", "nc_rural_S", "nc_outros_S"]].sum(axis=1)
)

# Totais nacionais (somando todas as regiões)
df_pivot["ec_total_BR"] = df_pivot[["ec_total_N", "ec_total_NE", "ec_total_CO", "ec_total_SE", "ec_total_S"]].sum(axis=1)
df_pivot["nc_total_BR"] = df_pivot[["nc_total_N", "nc_total_NE", "nc_total_CO", "nc_total_SE", "nc_total_S"]].sum(axis=1)

#================================================================================
# salvar em um arquivo Excel
#================================================================================
df_pivot.to_excel("dados/consumo_energia_pronto.xlsx", index=False)

#================================================================================
# converter para dataframe e converter a coluna data para datetime
#================================================================================
df = pd.DataFrame(df_pivot) 

# Converter coluna de data para datetime
df["Data"] = pd.to_datetime(df["Data"].astype(str), format="%Y%m%d")

df

,Data,nc_comercial_CO,nc_industrial_CO,nc_outros_CO,nc_residencial_CO,nc_rural_CO,nc_comercial_N,nc_industrial_N,nc_outros_N,nc_residencial_N,...,ec_total_NE,nc_total_NE,ec_total_CO,nc_total_CO,ec_total_SE,nc_total_SE,ec_total_S,nc_total_S,ec_total_BR,nc_total_BR
0,2004-01-01,342130,33512,34487,3210018,237641,238651,15267,32305,2292409,...,4457112.268,12699728,1507173.816,3857788,1.492953e+07,25582710,4510112.985,8511827,2.697836e+07,53331470
1,2004-02-01,341521,33368,34775,3217172,238298,244446,15355,33974,2350587,...,4139866.995,12738562,1456012.780,3865134,1.471471e+07,25678774,4690227.851,8531652,2.650827e+07,53560408
2,2004-03-01,341595,33362,34939,3230055,239170,242008,15274,32608,2341931,...,4372743.083,12789502,1534843.526,3879121,1.490380e+07,25794878,4898770.435,8552840,2.728434e+07,53751329
3,2004-04-01,341669,33439,35149,3241571,239969,248972,15426,34310,2404355,...,4461999.469,12842545,1610686.652,3891797,1.550132e+07,25822387,4873311.121,8569910,2.804618e+07,53934318
4,2004-05-01,342261,33352,35290,3252477,240787,240931,15239,33374,2330981,...,4423584.673,12814980,1565788.144,3904167,1.490663e+07,25827060,4723026.189,8568859,2.724816e+07,53841586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,2025-03-01,500035,31284,63938,6500483,501633,374325,11405,64744,5438905,...,8352240.741,25532335,3857464.839,7597373,2.375339e+07,40921083,9749096.477,14092360,4.922780e+07,94426391
255,2025-04-01,500066,31301,64818,6515735,500447,374354,11369,66776,5529522,...,8442959.511,25681297,3852413.357,7612367,2.232850e+07,40942785,8743640.909,14111760,4.699184e+07,94793908
256,2025-05-01,500623,31208,65221,6535656,502102,374787,11297,65070,5464130,...,8698664.676,25752113,3747947.641,7634810,2.197087e+07,41042453,8418626.627,14136616,4.657367e+07,94872932
257,2025-06-01,499689,31115,65448,6549028,502628,374918,11282,64239,5476536,...,8266953.109,25686775,3630921.308,7647908,2.113618e+07,40981602,8255491.329,14141718,4.497174e+07,94775579


-------------
#### Plotar Energia Consumida e Número de Consumidores — Brasil

In [26]:
# Variáveis que você quer visualizar
variables = ['ec_total_BR', 'nc_total_BR']

# Criar subplots (linhas = nº de variáveis, 1 coluna)
fig = make_subplots(
    rows=len(variables),
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=variables
)

# Adicionar cada série ao subplot
for i, var in enumerate(variables, start=1):
    fig.add_trace(
        go.Scatter(
            x=df["Data"],   # usa a coluna Data
            y=df[var],
            mode="lines",
            name=var,
            line=dict(color="steelblue", width=2)
        ),
        row=i, col=1
    )

# Layout
fig.update_layout(
    height=300*len(variables),
    width=1200,
    title_text="Análise Exploratória das Séries Temporais",
    showlegend=False,
    template="plotly_white"
)

# Mostrar
fig.show()


-------------
#### Exemplo aplicando Crhonos para prever o consumo de energia total do Brasil

----------------
#### Modelos pretrained

| Model              | Parameters | Based on            | Observações                                                                 |
|--------------------|------------|---------------------|------------------------------------------------------------------------------|
| chronos-t5-tiny    | 8M         | t5-efficient-tiny   | Modelo leve, útil para testes rápidos, séries curtas e dispositivos limitados |
| chronos-t5-mini    | 20M        | t5-efficient-mini   | Bom equilíbrio entre leveza e capacidade; adequado para prototipagem          |
| chronos-t5-small   | 46M        | t5-efficient-small  | Modelo recomendado para aplicações gerais em séries univariadas               |
| chronos-t5-base    | 200M       | t5-efficient-base   | Maior capacidade, melhor desempenho em séries longas, mais custo computacional|
| chronos-t5-large   | 710M       | t5-efficient-large  | Mais potente, mas pesado; indicado para grandes volumes de dados              |
| chronos-bolt-tiny  | 9M         | t5-efficient-tiny   | Versão Bolt otimizada; aceita covariadas; indicado para séries pequenas       |
| chronos-bolt-mini  | 21M        | t5-efficient-mini   | Suporta variáveis adicionais, bom custo-benefício                             |
| chronos-bolt-small | 48M        | t5-efficient-small  | Modelo Bolt mais usado; robusto para multivariadas e covariadas               |
| chronos-bolt-base  | 205M       | t5-efficient-base   | Melhor desempenho multivariado; adequado para previsões complexas             |


#### Exemplo Simples de aplicação do Chronos

In [27]:
#-------------------------------------------------------
# Dividir dados em treino e teste
#-------------------------------------------------------
# Definir datas de corte
data_fim = df["Data"].max()
data_corte = data_fim - pd.DateOffset(months=12)

# Criar treino e teste como pandas.DataFrame
df_treino = pd.DataFrame(df[df["Data"] <= data_corte]).reset_index(drop=True)
df_teste  = pd.DataFrame(df[df["Data"] > data_corte]).reset_index(drop=True)

#-------------------------------------------------------
# Carregar pipeline
#-------------------------------------------------------
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base",   # pode trocar por "amazon/chronos-bolt-small"
    device_map="cpu",            # use "cuda" se tiver GPU disponível
    torch_dtype=torch.float32    # float32 no CPU; em GPU pode usar bfloat16
)

#-------------------------------------------------------
# Garantias básicas
#-------------------------------------------------------
assert "Data" in df_treino.columns and "ec_total_BR" in df_treino.columns
assert "Data" in df_teste.columns and "ec_total_BR" in df_teste.columns

#-------------------------------------------------------
# Contexto: série histórica de treino (sem NaN)
#-------------------------------------------------------
y_hist = df_treino["ec_total_BR"].dropna().values.astype("float32")
context = torch.from_numpy(y_hist).unsqueeze(0)   # [1, time]

#-------------------------------------------------------
# Horizonte: tamanho do conjunto de teste (ex.: 12 meses)
#-------------------------------------------------------
horizon = len(df_teste)
future_index = df_teste["Data"].values  # datas-alvo

#-------------------------------------------------------
# Previsão (quantis)
#-------------------------------------------------------
# Dica: se horizon > 64, use a função em blocos que já te passei.
quantiles, mean = pipeline.predict_quantiles(
    context=context,
    prediction_length=horizon,
    quantile_levels=[0.1, 0.5, 0.9]
)
p10 = quantiles[0, :, 0].detach().cpu().numpy()
p50 = quantiles[0, :, 1].detach().cpu().numpy()
p90 = quantiles[0, :, 2].detach().cpu().numpy()

#-------------------------------------------------------
# DataFrame de previsões alinhado às datas do teste
#-------------------------------------------------------
df_forecast = pd.DataFrame({
    "Data": future_index,
    "ec_prev_p10": p10,
    "ec_prev_p50": p50,   # mediana = ponto de melhor chute
    "ec_prev_p90": p90
})

#-------------------------------------------------------
#  Avaliar no período de teste se tiver valor real
#-------------------------------------------------------
if df_teste["ec_total_BR"].notna().all():
    y_true = df_teste["ec_total_BR"].values.astype("float32")
    y_pred = p50.astype("float32")

    mae  = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(1e-9, y_true))) * 100

    print(f"\nMétricas no TESTE ({horizon} meses):")
    print(f"MAE : {mae:,.2f}")
    print(f"RMSE: {rmse:,.2f}")
    print(f"MAPE: {mape:.2f}%")

#-------------------------------------------------------
# Juntar previsões ao df_teste
#-------------------------------------------------------
df_teste_pred = df_teste.merge(df_forecast, on="Data", how="left")

#-------------------------------------------------------
# plotar projeções
#-------------------------------------------------------
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_treino["Data"], y=df_treino["ec_total_BR"],
                         mode="lines", name="Treino (real)"))
fig.add_trace(go.Scatter(x=df_teste["Data"], y=df_teste["ec_total_BR"],
                         mode="lines", name="Teste (real)"))
fig.add_trace(go.Scatter(x=df_forecast["Data"], y=df_forecast["ec_prev_p50"],
                         mode="lines", name="Previsão (P50)"))
fig.add_trace(go.Scatter(
    x=np.concatenate([df_forecast["Data"], df_forecast["Data"][::-1]]),
    y=np.concatenate([df_forecast["ec_prev_p90"], df_forecast["ec_prev_p10"][::-1]]),
    fill='toself', name="Intervalo 80%", opacity=0.25, line=dict(width=0)
))
fig.update_layout(title="ec_total_BR — Treino/Teste vs. Chronos",
                  template="plotly_white", xaxis_title="Data", yaxis_title="MWh")
fig.show()


Métricas no TESTE (12 meses):
MAE : 899,723.69
RMSE: 1,155,268.75
MAPE: 1.94%


--------------------------------------------------------------------

## Função para realizar o forecast com Chronos

#### Objetivo

Esta função **simplifica e padroniza** a projeção univariada com **Chronos (T5/Bolt)**. Com poucos *inputs* (dados, modelos e parâmetros básicos), ela executa todo o fluxo e devolve:

- **Gráfico interativo (Plotly)** com:
  - série histórica (treino e teste),
  - **P50** (mediana) de cada modelo,
  - **intervalos de projeção** (mesma cor da linha, em transparência),
  - período de teste **sombreado** para leitura rápida.
- **Métricas de erro** no período de teste (MAE, RMSE, MAPE, MedAE, R²).
- **DataFrame consolidado** (`df_forecast`) com o **df original + linhas futuras** e **colunas de projeção** por modelo (`__pL/__p50/__pU`), além de **ensemble** opcional.
- Objetos auxiliares: `forecasts` (dict por modelo), `df_treino`, `df_teste`.

**O que a função espera de entrada**
- Um `DataFrame` com a coluna de **data** (`date_col`, padrão `"Data"`) e a **coluna-alvo** (`target_col`, padrão `"ec_total_BR"`), preferencialmente em **frequência mensal (MS)** e **ordenado**.
- Uma lista `models` com identificadores Chronos:
  - **Online (Hugging Face):** nomes **com “/”** (ex.: `"amazon/chronos-bolt-small"`).
  - **Offline (local):** nomes **sem “/”** (ex.: `"amazon_chronos-bolt-small"`) + `local_base_dir`.
- Parâmetros opcionais de previsão: `n_periods` (horizonte), `quantile_levels` *ou* `interval_alpha`, `device` (`"cpu"`/`"cuda"`), `random_seed`, `ensemble`, etc.

**Diferenciais**
- **Horizonte longo > 64**: previsão em **blocos** encadeados automaticamente.
- **Quantis customizáveis** (ex.: `[0.1,0.5,0.9]`), com **ajuste automático** aos limites do **Bolt** (`[0.1, 0.9]`).
- **Cores consistentes** por modelo (intervalo = mesma cor translúcida; legenda de intervalo **sem valores** de quantil).
- **Ensemble** embutido (`mean`, `median`, `weighted`) para reduzir variância/sesgo.
- **Sem dor de cabeça de hardware**: usa GPU se disponível; caso contrário, cai para CPU.
- **Reprodutibilidade**: `random_seed` fixa NumPy/Random/Torch (quando aplicável).

**Limitações (intencionais)**
- **Univariada**: não usa covariáveis exógenas (foco na robustez e simplicidade).
- **Covariadas**: para rodar modelos com variáveis explicativas sugiro utilizar o package auto.gluon que implementa o Chronos com explicativas https://auto.gluon.ai/stable/index.html
- Incerteza **aumenta** naturalmente em horizontes muito longos.

> Resultado: um pipeline **fim-a-fim** e **opiniado** para comparar modelos Chronos, medir desempenho recente e obter projeções com **intervalos confiáveis**, pronto para integrar em relatórios/BI.


------------------
## Parâmetros

- **`df: pd.DataFrame`**  
  **O que é:** sua série histórica.  
  **Espera:** conter as colunas `date_col` (datas) e `target_col` (alvo numérico). Frequência **mensal** e ordenado por data.  
  **Por quê:** o modelo condiciona no histórico para prever; sem datas/valores válidos não há como gerar o contexto.  
  **Padrão:** — (obrigatório).

- **`models: list[str]`**  
  **O que é:** lista de identificadores de modelos Chronos.  
  **Espera:**  
  - **Online (Hugging Face):** nomes **com “/”** (ex.: `"amazon/chronos-bolt-small"`).  
  - **Offline (pasta local):** nomes **sem “/”** (ex.: `"amazon_chronos-bolt-small"`) + `local_base_dir` apontando para a raiz.  
  **Por quê:** a função carrega múltiplos modelos para comparar/ensemblar.  
  **Padrão:** — (obrigatório).

- **`target_col: str = "ec_total_BR"`**  
  **O que é:** nome da coluna alvo em `df`.  
  **Espera:** série numérica (float/int) com pelo menos `test_periods` pontos **não nulos**.  
  **Por quê:** define o que será previsto.  
  **Padrão:** `"ec_total_BR"`.

- **`test_periods: int = 12`**  
  **O que é:** quantos últimos pontos usar como **teste** (validação temporal).  
  **Espera:** `>= 1`.  
  **Por quê:** permite medir o erro no período mais recente e comparar modelos.  
  **Padrão:** `12` (12 meses).

- **`n_periods: int | None = None`**  
  **O que é:** horizonte de previsão (número de meses futuros).  
  **Espera:** `int > 0`; se `None`, usa `test_periods`.  
  **Por quê:** controla a extensão da projeção.  
  **Padrão:** `None` → vira `test_periods`.

- **`device: str = "cpu"`**  
  **O que é:** dispositivo alvo.  
  **Espera:** `"cpu"` ou `"cuda"`; se `"cuda"` mas GPU indisponível, faz **fallback** para CPU.  
  **Por quê:** performance (GPU acelera).  
  **Padrão:** `"cpu"`.

- **`force_offline: bool = False`**  
  **O que é:** força modo **offline** (sem internet) ao carregar modelos locais.  
  **Espera:** usado apenas quando `models` **não** têm “/” e `local_base_dir` foi informado.  
  **Por quê:** ambientes sem rede/reprodutibilidade.  
  **Padrão:** `False`.

- **`date_col: str = "Data"`**  
  **O que é:** nome da coluna de datas.  
  **Espera:** conversível para `datetime64`; a função tenta `YYYYMMDD` e, se falhar, `pd.to_datetime(..., errors="coerce")`.  
  **Por quê:** indexa o tempo e constrói o eixo futuro mensal (`freq="MS"`).  
  **Padrão:** `"Data"`.

- **`interval_alpha: float = 0.95`**  
  **O que é:** nível do intervalo quando **`quantile_levels` não é fornecido**.  
  **Espera:** número entre `(0,1)`; `0.95` → intervalos alvo **[2.5%, 97.5%]** (com mediana 50%).  
  **Por quê:** controlar a **incerteza** exibida.  
  **Padrão:** `0.95`.

- **`quantile_levels: list[float] | None = None`**  
  **O que é:** lista explícita de quantis (ex.: `[0.1, 0.5, 0.9]`).  
  **Espera:** incluir `0.5` (mediana); a função ordena e **fixa a mediana** em 0.5.  
  **Por quê:** personalizar o intervalo de projeção.  
  **Observação:** para **BOLT**, quantis são **ajustados** ao intervalo suportado `[0.1, 0.9]`.  
  **Padrão:** `None` → derivado de `interval_alpha`.

- **`local_base_dir: str | None = None`**  
  **O que é:** pasta raiz dos modelos locais.  
  **Espera:** subpastas por modelo contendo `config.json` e pesos.  
  **Por quê:** necessário no modo offline (nomes de modelo **sem “/”**).  
  **Padrão:** `None`.

- **`color_map: dict[str, str] | None = None`**  
  **O que é:** dicionário `label_do_modelo → cor` (hex/nome).  
  **Espera:** chaves iguais aos **labels** usados no gráfico (sufixo para online, nome bruto para offline).  
  **Por quê:** garantir **cores consistentes** para P50 e seus intervalos (intervalo = mesma cor translúcida).  
  **Padrão:** `None` → paleta automática.

- **`random_seed: int | None = 123`**  
  **O que é:** semente para reprodutibilidade (NumPy/Random/Torch).  
  **Espera:** `int` ou `None` (para não fixar).  
  **Por quê:** resultados estáveis entre execuções.  
  **Padrão:** `123`.

- **`allow_long_horizon: bool = True`**  
  **O que é:** habilita previsão **em blocos** quando `n_periods > 64`.  
  **Espera:** `True/False`.  
  **Por quê:** contorna o limite prático de comprimento prevendo pedaços de até 64 e encadeando via P50.  
  **Padrão:** `True`.

- **`ensemble: dict | None = None`**  
  **O que é:** instruções para combinar modelos.  
  **Espera:** dicionário com chaves:  
  - `"method"`: `"mean" | "median" | "weighted"`  
  - `"weights"`: `{label: peso, ...}` (apenas se `"weighted"`)  
  - `"models"`: `list[str]` com os labels a combinar (opcional; se ausente, usa todos)  
  **Por quê:** reduzir variância/sesgo combinando previsões.  
  **Padrão:** `None` (sem ensemble).

---

### Regras de validação importantes
- `df[target_col]` deve ter **≥ `test_periods`** valores não nulos (senão a função lança erro).  
- Modelos **online** exigem internet/cachê; **offline** exigem `local_base_dir` válido.  
- Se `quantile_levels=None`, os quantis são derivados de `interval_alpha`; para **BOLT** sempre respeitando `[0.1, 0.9]`.




### Função para forecast com Chronos

In [28]:
def function_chronos_forecast(
    df: pd.DataFrame,
    models: list[str],
    target_col: str = "ec_total_BR",
    test_periods: int = 12,
    n_periods: int | None = None,      # horizonte desejado
    device: str = "cpu",
    force_offline: bool = False,
    date_col: str = "Data",
    interval_alpha: float = 0.95,      # usado se quantile_levels=None
    quantile_levels: list[float] | None = None,  # ex.: [0.1, 0.5, 0.9]
    local_base_dir: str | None = None,
    color_map: dict[str, str] | None = None,
    random_seed: int | None = 123,
    allow_long_horizon: bool = True,   # se >64, faz blocos e concatena
    ensemble: dict | None = None,      # {"method": "mean"|"median"|"weighted", "weights": {"label": w, ...}, "models": ["label", ...]}
):
    """
    Retorna:
      fig (Plotly), metrics_df (sem Ident/PinballLoss), forecasts (dict[label]->DF com Data+pL/p50/pU),
      df_treino, df_teste, df_forecast (df original + colunas de projeção por modelo e, se definido, ensemble_*).

    Observações:
    - Modelos com "/" são online (Hugging Face); sem "/" => pasta local em local_base_dir.
    - BOLT aceita quantis em [0.1, 0.9]; ajustamos automaticamente se precisar.
    - Intervalos usam a MESMA cor do modelo com transparência. A legenda do intervalo NÃO mostra valores dos quantis.
    """
 
    # ---------------- Seeds (determinismo prático) ----------------
    if random_seed is not None:
        np.random.seed(random_seed)
        random.seed(random_seed)
        try:
            torch.manual_seed(random_seed)
            torch.use_deterministic_algorithms(True)
        except Exception:
            pass  # em alguns builds essa flag pode não estar disponível

    # ---------------- Paleta e utilitários ----------------
    default_palette = [
        "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728",
        "#9467bd", "#8c564b", "#e377c2", "#7f7f7f",
        "#bcbd22", "#17becf"
    ]
    if color_map is None:
        color_map = {
            "amazon_chronos-t5-tiny":  "#2ca02c",
            "amazon_chronos-t5-mini":  "#1f77b4",
            "amazon_chronos-t5-small": "#ff7f0e",
            "amazon_chronos-t5-base":  "#9467bd",
            "amazon_chronos-t5-large": "#d62728",
            "amazon_chronos-bolt-tiny":  "#17becf",
            "amazon_chronos-bolt-mini":  "#bcbd22",
            "amazon_chronos-bolt-small": "#8c564b",
            "amazon_chronos-bolt-base":  "#e377c2",
        }
    used_colors = {}
    def get_color_for_label(label: str) -> str:
        if label in color_map:
            return color_map[label]
        for c in default_palette:
            if c not in used_colors.values():
                return c
        return "#7f7f7f"

    # ---------------- Preparação do DF ----------------
    df = df.copy()
    if date_col not in df.columns:
        raise ValueError(f"Coluna de data '{date_col}' não encontrada.")
    if target_col not in df.columns:
        raise ValueError(f"Coluna alvo '{target_col}' não encontrada.")

    if not np.issubdtype(df[date_col].dtype, np.datetime64):
        try:
            df[date_col] = pd.to_datetime(df[date_col].astype(str), format="%Y%m%d", errors="coerce")
        except Exception:
            df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.sort_values(date_col).reset_index(drop=True)

    non_missing_idx = df[df[target_col].notna()].index
    if len(non_missing_idx) < max(1, test_periods):
        raise ValueError(f"Não há {test_periods} pontos não-nulos em '{target_col}' para formar o teste.")

    test_idx = non_missing_idx[-test_periods:]
    train_idx = non_missing_idx[:-test_periods]
    df_treino = df.loc[train_idx, [date_col, target_col]].reset_index(drop=True)
    df_teste  = df.loc[test_idx,  [date_col, target_col]].reset_index(drop=True)

    # Horizonte desejado
    if n_periods is None:
        n_periods = test_periods
    n_periods = int(n_periods)

    # Índices temporais de previsão
    start_date = df_teste[date_col].iloc[0] if len(df_teste) > 0 else (df_treino[date_col].max() + pd.offsets.MonthBegin(1))
    future_index_full = pd.date_range(start=start_date, periods=n_periods, freq="MS")

    # Interseção p/ métricas
    overlap_len = min(n_periods, len(df_teste))
    df_teste_used = df_teste.iloc[:overlap_len].copy()
    shade_start = df_teste_used[date_col].iloc[0] if overlap_len > 0 else None
    shade_end   = df_teste_used[date_col].iloc[-1] if overlap_len > 0 else None

    # ---------------- Device ----------------
    use_device = "cuda" if (device.lower() == "cuda" and torch.cuda.is_available()) else "cpu"
    if device.lower() == "cuda" and not torch.cuda.is_available():
        print("[AVISO] CUDA não disponível. Usando CPU.")

    # ---------------- Gráfico base ----------------
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_treino[date_col], y=df_treino[target_col],
        mode="lines", name="Treino (real)",
        line=dict(color="black", width=2)
    ))
    fig.add_trace(go.Scatter(
        x=df_teste[date_col], y=df_teste[target_col],
        mode="lines", name="Teste (real)",
        line=dict(color="#3366cc", width=2)
    ))
    if overlap_len > 0:
        fig.add_shape(
            type="rect", xref="x", yref="paper",
            x0=shade_start, x1=shade_end, y0=0, y1=1,
            fillcolor="rgba(0,0,0,0.07)", line=dict(width=0), layer="below"
        )

    # ---------------- Métricas (enxutas) ----------------
    metrics_cols = [
        "Label","Fonte","Variável","Previsto","Avaliado_sobre",
        "MAE","RMSE","MAPE (%)","MedAE","R2","Lower_q","Upper_q"
    ]
    metrics_rows = []
    forecasts = {}

    # ---------------- Contexto treino ----------------
    y_hist = df_treino[target_col].dropna().values.astype("float32")
    context_base = torch.from_numpy(y_hist).unsqueeze(0)  # [1, T]

    # ---------------- Quantis desejados ----------------
    def build_quantiles_for_model(model_id: str) -> tuple[list[float], float, float]:
        # Se vier explicitamente, usa; senão, deriva de interval_alpha
        if quantile_levels:
            q_sorted = sorted(quantile_levels)
        else:
            lo = (1.0 - interval_alpha) / 2.0
            hi = 1.0 - lo
            q_sorted = [lo, 0.5, hi]
        # Ajuste para BOLT (limite [0.1,0.9])
        lo, mid, hi = q_sorted[0], 0.5, q_sorted[-1]
        if "bolt" in model_id.lower():
            lo = max(0.1, min(0.9, lo))
            hi = max(0.1, min(0.9, hi))
            if lo >= mid: lo = 0.1
            if hi <= mid: hi = 0.9
            q_sorted = [lo, mid, hi]
        return q_sorted, q_sorted[0], q_sorted[-1]

    # ---------------- df_forecast consolidado ----------------
    df_forecast = df.copy()
    last_hist_date = df_forecast[date_col].max()
    extra_dates = [d for d in future_index_full if d > last_hist_date]
    if extra_dates:
        df_extra = pd.DataFrame({date_col: extra_dates})
        for c in df_forecast.columns:
            if c != date_col:
                df_extra[c] = np.nan
        df_forecast = pd.concat([df_forecast, df_extra], ignore_index=True, sort=False)
    if f"{target_col}_real" not in df_forecast.columns:
        df_forecast[f"{target_col}_real"] = df_forecast[target_col]

    # ---------------- Previsão em blocos (>64) ----------------
    def predict_in_blocks(pipe, ctx_tensor, pred_len, q_levels):
        """
        Divide em blocos de 64 (limite prático) se necessário e concatena.
        ctx_tensor: torch tensor [1, T]
        Retorna: numpy array [pred_len, len(q_levels)]
        """
        max_block = 64
        remaining = pred_len
        y_ctx = ctx_tensor.clone()
        out_list = []
        while remaining > 0:
            step = min(max_block, remaining)
            qs, _ = pipe.predict_quantiles(
                context=y_ctx, prediction_length=step, quantile_levels=q_levels
            )
            np_block = qs[0].detach().cpu().numpy()  # [step, len(q_levels)]
            out_list.append(np_block)
            # atualiza contexto com mediana (p50) para seguir
            p50_idx = q_levels.index(0.5)
            p50_vals = qs[0, :, p50_idx].detach().cpu()
            y_ctx = torch.cat([y_ctx, p50_vals.unsqueeze(0)], dim=1)
            remaining -= step
        return np.concatenate(out_list, axis=0)

    # ---------------- Loop de modelos ----------------
    for model_str in models:
        is_online = ("/" in model_str)
        label = model_str.split("/")[-1] if is_online else model_str

        # Carregar modelo
        try:
            if is_online:
                os.environ.pop("TRANSFORMERS_OFFLINE", None)
                pipe = BaseChronosPipeline.from_pretrained(
                    model_str, device_map=None, dtype=torch.float32, local_files_only=False
                )
                fonte = "huggingface"
                ident_for_quant = model_str
            else:
                if not local_base_dir:
                    raise ValueError("Modelo offline informado sem 'local_base_dir'.")
                model_dir = os.path.join(local_base_dir, model_str)
                if not (os.path.isdir(model_dir) and os.path.isfile(os.path.join(model_dir, "config.json"))):
                    raise FileNotFoundError(f"Pasta do modelo local não encontrada ou inválida: {model_dir}")
                if force_offline:
                    os.environ["TRANSFORMERS_OFFLINE"] = "1"
                pipe = BaseChronosPipeline.from_pretrained(
                    model_dir, local_files_only=True, device_map=None, dtype=torch.float32
                )
                fonte = "local"
                ident_for_quant = model_dir
            try:
                pipe.model.to(use_device)
            except Exception:
                pipe.model.to("cpu")
        except Exception as e:
            print(f"[ERRO] Falha ao carregar modelo '{model_str}': {e}")
            metrics_rows.append([
                label, ("huggingface" if is_online else "local"), target_col,
                n_periods, overlap_len, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
            ])
            continue

        # Quantis
        q_levels, lower_q, upper_q = build_quantiles_for_model(ident_for_quant)

        # Prever (blocos se necessário)
        try:
            if allow_long_horizon and n_periods > 64:
                pvals = predict_in_blocks(pipe, context_base, n_periods, q_levels)
            else:
                qs, _ = pipe.predict_quantiles(
                    context=context_base, prediction_length=n_periods, quantile_levels=q_levels
                )
                pvals = qs[0].detach().cpu().numpy()  # [n_periods, len(q_levels)]
        except Exception as e:
            print(f"[ERRO] Falha ao prever com '{label}': {e}")
            metrics_rows.append([
                label, fonte, target_col, n_periods, overlap_len,
                np.nan, np.nan, np.nan, np.nan, np.nan, lower_q, upper_q
            ])
            continue

        # Montar DF de forecast do modelo
        q_lo_idx, q_md_idx, q_hi_idx = 0, q_levels.index(0.5), len(q_levels)-1
        df_fc_model = pd.DataFrame({
            date_col: future_index_full,
            f"{target_col}__{label}__pL": pvals[:, q_lo_idx],
            f"{target_col}__{label}__p50": pvals[:, q_md_idx],
            f"{target_col}__{label}__pU": pvals[:, q_hi_idx],
        })
        forecasts[label] = df_fc_model

        # Métricas na interseção com teste
        if overlap_len > 0 and df_teste_used[target_col].notna().any():
            y_true = df_teste_used[target_col].astype("float64").values
            y_p50  = df_fc_model[f"{target_col}__{label}__p50"].iloc[:overlap_len].astype("float64").values
            mae   = mean_absolute_error(y_true, y_p50)
            rmse  = np.sqrt(mean_squared_error(y_true, y_p50))
            mape  = mean_absolute_percentage_error(y_true, y_p50) * 100.0
            medae = median_absolute_error(y_true, y_p50)
            r2    = r2_score(y_true, y_p50)
        else:
            mae=rmse=mape=medae=r2=np.nan

        metrics_rows.append([
            label, fonte, target_col, n_periods, overlap_len,
            mae, rmse, mape, medae, r2, lower_q, upper_q
        ])

        # Plot: cor consistente e intervalo com transparência
        base_color = get_color_for_label(label)
        used_colors[label] = base_color

        fig.add_trace(go.Scatter(
            x=df_fc_model[date_col],
            y=df_fc_model[f"{target_col}__{label}__p50"],
            mode="lines",
            name=f"P50 — {label}",
            line=dict(width=2, color=base_color)
        ))
        fig.add_trace(go.Scatter(
            x=np.concatenate([df_fc_model[date_col], df_fc_model[date_col][::-1]]),
            y=np.concatenate([
                df_fc_model[f"{target_col}__{label}__pU"],
                df_fc_model[f"{target_col}__{label}__pL"][::-1]
            ]),
            fill='toself',
            name=f"Intervalo — {label}",
            line=dict(width=0),
            fillcolor=base_color,
            opacity=0.20
        ))

        # Mesclar previsões no consolidado
        df_forecast = df_forecast.merge(df_fc_model, on=date_col, how="left")

    # ---------------- DataFrame de métricas ----------------
    metrics_df = pd.DataFrame(metrics_rows, columns=metrics_cols)

    # ---------------- Ensemble (opcional) ----------------
    if ensemble is not None:
        method = ensemble.get("method", "mean")
        weights = ensemble.get("weights", {})
        models_for_ens = ensemble.get("models", list(forecasts.keys()))

        # coletores por tipo de quantil
        cols_pL = [f"{target_col}__{m}__pL" for m in models_for_ens if f"{target_col}__{m}__pL" in df_forecast.columns]
        cols_p50 = [f"{target_col}__{m}__p50" for m in models_for_ens if f"{target_col}__{m}__p50" in df_forecast.columns]
        cols_pU = [f"{target_col}__{m}__pU" for m in models_for_ens if f"{target_col}__{m}__pU" in df_forecast.columns]

        def combine(cols, qname):
            if not cols:
                return
            if method == "mean":
                df_forecast[f"{target_col}__ensemble__{qname}"] = df_forecast[cols].mean(axis=1)
            elif method == "median":
                df_forecast[f"{target_col}__ensemble__{qname}"] = df_forecast[cols].median(axis=1)
            elif method == "weighted":
                # normaliza pesos só para as colunas presentes
                ws = np.array([weights.get(c.split("__")[1], 0.0) for c in cols], dtype="float64")
                if ws.sum() == 0:
                    df_forecast[f"{target_col}__ensemble__{qname}"] = df_forecast[cols].mean(axis=1)
                else:
                    ws = ws / ws.sum()
                    df_forecast[f"{target_col}__ensemble__{qname}"] = (df_forecast[cols].values * ws).sum(axis=1)
            else:
                df_forecast[f"{target_col}__ensemble__{qname}"] = df_forecast[cols].mean(axis=1)

        combine(cols_pL, "pL")
        combine(cols_p50, "p50")
        combine(cols_pU, "pU")

        # plota ensemble (se existir p50)
        ens_col = f"{target_col}__ensemble__p50"
        if ens_col in df_forecast.columns:
            ens_color = "#000000"
            fig.add_trace(go.Scatter(
                x=df_forecast[date_col],
                y=df_forecast[ens_col],
                mode="lines",
                name="P50 — ensemble",
                line=dict(width=3, color=ens_color, dash="dash")
            ))
            # intervalo do ensemble (se existir)
            if f"{target_col}__ensemble__pL" in df_forecast.columns and f"{target_col}__ensemble__pU" in df_forecast.columns:
                fig.add_trace(go.Scatter(
                    x=np.concatenate([df_forecast[date_col], df_forecast[date_col][::-1]]),
                    y=np.concatenate([
                        df_forecast[f"{target_col}__ensemble__pU"],
                        df_forecast[f"{target_col}__ensemble__pL"][::-1]
                    ]),
                    fill='toself',
                    name="Intervalo — ensemble",
                    line=dict(width=0),
                    fillcolor=ens_color,
                    opacity=0.12
                ))

    # ---------------- Layout final ----------------
    title_extra = f" | Avaliado (sombreado): {shade_start.date()} → {shade_end.date()}" if overlap_len > 0 else ""
    fig.update_layout(
        title=f"Forecast {target_col} — {len(models)} modelos | Previsto: {n_periods} meses{title_extra}",
        template="plotly_white",
        xaxis_title="Data",
        yaxis_title=target_col,
        height=820,
        legend=dict(orientation="v", x=1.02, xanchor="left", y=1),
        margin=dict(l=60, r=200, t=80, b=60)
    )

    return fig, metrics_df, forecasts, df_treino, df_teste, df_forecast


### Rodar modelos **offline**

Para evitar downloads repetidos e acelerar os experimentos, você pode rodar os modelos **localmente** (offline).  
No notebook `Download_amazon_cronos_models.ipynb` eu baixo e organizo os pesos em disco. Tamanhos típicos:

- `amazon_chronos-bolt-base`  ≈ **783 MB**
- `amazon_chronos-bolt-small` ≈ **182 MB**
- `amazon_chronos-bolt-mini`  ≈ **81 MB**
- `amazon_chronos-bolt-tiny`  ≈ **33 MB**
- `amazon_chronos-t5-large`   ≈ **2.64 GB**
- `amazon_chronos-t5-base`    ≈ **768 MB**
- `amazon_chronos-t5-small`   ≈ **176 MB**
- `amazon_chronos-t5-mini`    ≈ **78 MB**
- `amazon_chronos-t5-tiny`    ≈ **32 MB**

> **Dica:** em ambientes **efêmeros** (ex.: Colab/Databricks com disco temporário), rodar **online** baixa os pesos **a cada sessão**. Em máquinas com disco persistente, o Hugging Face **cacheia** em `~/.cache/huggingface/hub`, evitando novo download.

#### Como organizar os modelos no disco
Estruture um diretório base (ex.: `D:/Models/chronos`) com **uma pasta por modelo**, contendo pelo menos `config.json` e os pesos:


In [ ]:
# Defindo a pasta local onde estão os modelos baixados
Path_models = Path(r"Models_cronos_amazon/cronos")

# Lista de modelos offline
models_offline = [
    "amazon_chronos-t5-tiny",
    "amazon_chronos-t5-small",
    "amazon_chronos-t5-mini",
    "amazon_chronos-t5-large",
    "amazon_chronos-t5-base",

    "amazon_chronos-bolt-tiny",
    "amazon_chronos-bolt-small",
    "amazon_chronos-bolt-mini",
    "amazon_chronos-bolt-base",
]

# Chamar a função
fig_off, met_off, fc_dict, df_treino, df_teste, df_forecast = function_chronos_forecast(
    df=df,                                  # Dataframe onde estão os dados (não é necessário dividir em treino e teste)
    models=models_offline,                  # roda todos os modelos listados (offline)
    target_col="ec_total_BR",               # coluna-alvo
    test_periods=12,                        # últimos 6 meses como teste (para métricas) esses meses não entram no treinamento
    n_periods=60,                           # horizonte de 60 meses
    device="cuda",                          # tenta GPU; se indisponível, costuma cair para CPU
    local_base_dir=str(Path_models),        # raiz das pastas dos modelos
    force_offline=True,                     # não baixa nada da internet
    quantile_levels=[0.1, 0.5, 0.9],        # define P10/P50/P90 como intervalo padrão ou use interval_alpha exemplo: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    random_seed=2025,                       # reprodutibilidade prática
    allow_long_horizon=True,                # ativa previsão em blocos (>64)
    ensemble={                              # Configurações do ensemble
        "method": "weighted",               # combina modelos com pesos
        "weights": {                        # pesos só para T5 (large/base/mini)
            "amazon_chronos-t5-large": 0.5, # 50%
            "amazon_chronos-t5-base":  0.3, # 30%
            "amazon_chronos-t5-mini":  0.2  # 20%
        },
        "models": [
            "amazon_chronos-t5-large",
            "amazon_chronos-t5-base",
            "amazon_chronos-t5-mini"
        ]
    }
)

# Plotar resultados
fig_off.show()


[AVISO] CUDA não disponível. Usando CPU.


In [35]:
## Métricas dos modelos treinados
met_off

,Label,Fonte,Variável,Previsto,Avaliado_sobre,MAE,RMSE,MAPE (%),MedAE,R2,Lower_q,Upper_q
0,amazon_chronos-t5-tiny,local,ec_total_BR,60,12,9.347551e+05,1.041152e+06,2.008194,794023.2880,0.168707,0.1,0.9
1,amazon_chronos-t5-small,local,ec_total_BR,60,12,1.016172e+06,1.271533e+06,2.183139,779752.8810,-0.239885,0.1,0.9
2,amazon_chronos-t5-mini,local,ec_total_BR,60,12,5.987633e+05,8.082555e+05,1.284735,354226.7925,0.499017,0.1,0.9
3,amazon_chronos-t5-large,local,ec_total_BR,60,12,5.697262e+05,6.359565e+05,1.225219,503865.1995,0.689844,0.1,0.9
4,amazon_chronos-t5-base,local,ec_total_BR,60,12,6.825362e+05,8.925196e+05,1.470787,444758.1920,0.389112,0.1,0.9
5,amazon_chronos-bolt-tiny,local,ec_total_BR,60,12,6.076691e+05,7.857927e+05,1.288521,442004.6480,0.526476,0.1,0.9
6,amazon_chronos-bolt-small,local,ec_total_BR,60,12,4.811372e+05,6.395806e+05,1.018405,381562.9285,0.686299,0.1,0.9
7,amazon_chronos-bolt-mini,local,ec_total_BR,60,12,5.755491e+05,7.671012e+05,1.213385,443312.8695,0.548735,0.1,0.9
8,amazon_chronos-bolt-base,local,ec_total_BR,60,12,5.786537e+05,7.095963e+05,1.226262,515204.6705,0.613857,0.1,0.9


### Resumo das métricas

- **Janela:** `Previsto = 60` meses; métricas calculadas nos **últimos 12** (`Avaliado_sobre = 12`).
- **Intervalos:** todas as previsões com **P10–P90** (`Lower_q = 0.1`, `Upper_q = 0.9`).

#### Destaques (melhores resultados)
- **amazon_chronos-bolt-small**  
  - **Menor MAPE** (~**1.02%**) e **menor MAE** (~**4.81e5**).  
  - **R² ≈ 0.686** → desempenho sólido e consistente.
- **amazon_chronos-t5-large**  
  - **Maior R²** (**~0.690**) e **menor RMSE** (~**6.36e5**).  
  - MAE baixo, porém é um modelo mais **pesado**.

#### Bons custos-benefícios
- **bolt-base / bolt-mini / bolt-tiny**  
  - MAPE ~**1.21–1.29%**, **R² ~0.53–0.61** → bons resultados vs. tamanho/velocidade.

#### Modelos abaixo do esperado
- **t5-small** e **t5-tiny**  
  - MAPE maiores (~**2.18%** / **2.01%**) e **R² baixo** (t5-small chegou a **R² negativo**).

#### Como ler as colunas
- **MAE / RMSE / MedAE:** quanto **menor**, melhor.  
- **MAPE (%):** erro percentual médio (comparável).  
- **R²:** quanto **mais perto de 1**, melhor (negativo = pior que baseline ingênua).

#### Sugestão prática
- Use **bolt-small** como **baseline**.  
- Se quiser ganho adicional, faça **ensemble ponderado** com **t5-large** (ex.: 60–70% *bolt_small* + 30–40% *t5_large*).


In [36]:
# Prévia do dataframe com as projeções
df_forecast.head()

,Data,nc_comercial_CO,nc_industrial_CO,nc_outros_CO,nc_residencial_CO,nc_rural_CO,nc_comercial_N,nc_industrial_N,nc_outros_N,nc_residencial_N,...,ec_total_BR__amazon_chronos-bolt-small__pU,ec_total_BR__amazon_chronos-bolt-mini__pL,ec_total_BR__amazon_chronos-bolt-mini__p50,ec_total_BR__amazon_chronos-bolt-mini__pU,ec_total_BR__amazon_chronos-bolt-base__pL,ec_total_BR__amazon_chronos-bolt-base__p50,ec_total_BR__amazon_chronos-bolt-base__pU,ec_total_BR__ensemble__pL,ec_total_BR__ensemble__p50,ec_total_BR__ensemble__pU
0,2004-01-01,342130.0,33512.0,34487.0,3210018.0,237641.0,238651.0,15267.0,32305.0,2292409.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-02-01,341521.0,33368.0,34775.0,3217172.0,238298.0,244446.0,15355.0,33974.0,2350587.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-03-01,341595.0,33362.0,34939.0,3230055.0,239170.0,242008.0,15274.0,32608.0,2341931.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-04-01,341669.0,33439.0,35149.0,3241571.0,239969.0,248972.0,15426.0,34310.0,2404355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-05-01,342261.0,33352.0,35290.0,3252477.0,240787.0,240931.0,15239.0,33374.0,2330981.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


--------------
### Rodar modelos online

Neste exemplo estaremos baixando os modelos antes de rodar.

In [ ]:
# Lista de modelos offline
models_online = [
    "amazon/chronos-t5-tiny",
    #"amazon/chronos-t5-small",
    #"amazon/chronos-t5-mini",
    #"amazon/chronos-t5-large",
    #"amazon/chronos-t5-base",

    "amazon/chronos-bolt-tiny",
    #"amazon/chronos-bolt-small",
    #"amazon/chronos-bolt-mini",
    #"amazon/chronos-bolt-base",
]

levels_quantile = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

fig_on, met_on, fc_dict, df_treino, df_teste, df_forecast = function_chronos_forecast(
    df=df,
    models=models_online,  
    target_col="ec_total_BR",
    test_periods=12,
    n_periods=60,          
    device="cuda",
    #local_base_dir=str(Path_models),  # Aqui comento o código pois não vamos utilizar os modelos locais
    force_offline=True,                # para rodar online essa opçõa deve ser True
    quantile_levels=[0.1, 0.5, 0.9],  
    random_seed=2025,
    allow_long_horizon=False,
    ensemble={"method": "mean"}        # Para exemplicar escolhi um ensemble com a média dos modelos treinados
)
fig_on.show()


[AVISO] CUDA não disponível. Usando CPU.


In [39]:
## Métricas dos modelos treinados
met_on

,Label,Fonte,Variável,Previsto,Avaliado_sobre,MAE,RMSE,MAPE (%),MedAE,R2,Lower_q,Upper_q
0,chronos-t5-tiny,huggingface,ec_total_BR,60,12,934755.05425,1.041152e+06,2.008194,794023.288,0.168707,0.1,0.9
1,chronos-bolt-tiny,huggingface,ec_total_BR,60,12,607669.12825,7.857927e+05,1.288521,442004.648,0.526476,0.1,0.9


In [40]:
# Prévia do dataframe com as projeções
df_forecast.head()

,Data,nc_comercial_CO,nc_industrial_CO,nc_outros_CO,nc_residencial_CO,nc_rural_CO,nc_comercial_N,nc_industrial_N,nc_outros_N,nc_residencial_N,...,ec_total_BR_real,ec_total_BR__chronos-t5-tiny__pL,ec_total_BR__chronos-t5-tiny__p50,ec_total_BR__chronos-t5-tiny__pU,ec_total_BR__chronos-bolt-tiny__pL,ec_total_BR__chronos-bolt-tiny__p50,ec_total_BR__chronos-bolt-tiny__pU,ec_total_BR__ensemble__pL,ec_total_BR__ensemble__p50,ec_total_BR__ensemble__pU
0,2004-01-01,342130.0,33512.0,34487.0,3210018.0,237641.0,238651.0,15267.0,32305.0,2292409.0,...,2.697836e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-02-01,341521.0,33368.0,34775.0,3217172.0,238298.0,244446.0,15355.0,33974.0,2350587.0,...,2.650827e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-03-01,341595.0,33362.0,34939.0,3230055.0,239170.0,242008.0,15274.0,32608.0,2341931.0,...,2.728434e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-04-01,341669.0,33439.0,35149.0,3241571.0,239969.0,248972.0,15426.0,34310.0,2404355.0,...,2.804618e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-05-01,342261.0,33352.0,35290.0,3252477.0,240787.0,240931.0,15239.0,33374.0,2330981.0,...,2.724816e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


--------------

## Conclusão

Este script entrega um **pipeline completo e reprodutível** para previsão univariada com **Chronos (T5/Bolt)**, desde a **preparação dos dados** até a **avaliação** e **visualização**:

- **Preparação dos dados:** padronização de regiões (siglas), normalização de classes, agregação de **Consumo** e **Consumidores** por `Data × Região × Classe`, *pivot* para formato amplo, criação de **totais regionais** e **total Brasil** — resultando em um `DataFrame` limpo, mensal e pronto para modelagem.
- **Modelagem:** função unificada que carrega múltiplos modelos (modo **online** via Hugging Face ou **offline** via diretório local), prevê **horizontes longos** com **blocos**, gera **quantis** (ex.: P10–P50–P90), aplica **cores consistentes** e calcula **métricas** no período de teste.
- **Resultados:** gráfico interativo (Plotly) com histórico (treino/teste), **P50** por modelo e **intervalos**; `metrics_df` com **MAE, RMSE, MAPE, MedAE, R²**; e `df_forecast` consolidando **dados originais + projeções** por modelo e **ensemble** opcional.
- **Desempenho observado:** modelos **Bolt** (especialmente `bolt-small`) mostraram **excelente custo-benefício**; `t5-large` obteve **R²** e **RMSE** muito fortes. Um **ensemble ponderado** entre ambos tende a estabilizar e melhorar a acurácia.
- **Operacionalização:** suporte a **modo offline** elimina latência de download e facilita rodar muitos modelos em lote; **seed** garante reprodutibilidade prática; estrutura de saída pronta para **exportar** (CSV/Excel) e integrar em BI.

**Limitações assumidas:** abordagem **univariada** (sem covariáveis exógenas) e, naturalmente, maior incerteza em horizontes muito longos — refletida pela abertura dos intervalos.

**Próximos passos sugeridos:** evoluir para **multivariado** (covariáveis) utilizando auto gluon

